In [ ]:
%matplotlib widget
import requests
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import time

In [ ]:
def get_day_weather(day, month, year):
    date = f'{year}-{month:02}-{day:02}'
    print(date)
    url = f"https://www.meteociel.fr/temps-reel/obs_villes.php?code2=6180&jour2={day}&mois2={month-1}&annee2={year}"
    r = requests.get(url)
    if r.status_code != 200:
        return [], []
    lines = r.text.split('\n')
    filtered = [line for line in lines if 'Direction' in line]
    
    wind_angles = []
    for line in filtered:
        degs = re.findall('\d{1,3}&deg;', line)
        if len(degs) == 0:
            return  [], []
        wind_angles.append(int(degs[0].split('&')[0]))
    wind_angles = wind_angles[::-1]
    
    times = []
    for line in filtered:
        t = re.findall('\d{1,2}h\d{1,2}', line)
        if len(t) == 0:
             return [], []
        # Extract hour and minute from the time string
        hour, minute = map(int, t[0].split('h'))
        # Combine date and time components into a single string
        combined_datetime_str = f'{date} {hour:02d}:{minute:02d}'
        # Convert the combined string to a NumPy datetime64 object
        dt = np.datetime64(combined_datetime_str)
        times.append(dt)
    times = np.array(times)[::-1]
    times
    
    return times, wind_angles

In [ ]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [ ]:
year = 2023

days_in_month = {"Jan": 31, "Feb": 28, "Mar": 31, "Apr": 30, "May": 31, "Jun": 30, "Jul": 31, "Aug": 31, "Sep": 30, "Oct": 31, "Nov": 30, "Dec": 31}

times = []
wind = []

months = list(days_in_month.keys())[:8]

for im, month in enumerate(months):
    for day in range(1, days_in_month[month] + 1):
        t, w = get_day_weather(day, im + 1, year)
        times.append(t)
        wind.append(w)
        time.sleep(0.3)
    

times = flatten(times)
wind = flatten(wind)

In [ ]:
# %matplotlib widget
fig, ax = plt.subplots()
ax.plot(np.array(times), np.array(wind))

In [ ]:
# Save raw data
np.save('wind', np.array(wind))
np.save('times', np.array(times))

0 -> -1

180 -> +1

90 -> 0

270 -> 0

In [ ]:
def func(x):
    b = 180.
    c = 20.
    return np.exp(-(x-b)**2 / (2.*(c**2)))

In [ ]:
# fig, ax = plt.subplots()
# x = np.linspace(0, 360, 1000)
# ax.plot(x, func(x))

In [ ]:
times = np.array(times)
wind = func(np.array(wind))
fig, ax = plt.subplots()
ax.plot(times, wind)

In [ ]:
# np.save('wind', wind)
# np.save('times', times)

In [ ]:
stations = ['backersvej', 'soetorvet']

air = {stat: pd.read_csv(f'{stat}.csv', delimiter=';') for stat in stations}

In [ ]:
for key in air:
    air[key] = air[key].loc[air[key]['MeanValueType'] == "1H"].copy()

    start = pd.to_datetime(air[key]['StartUtc'])
    end = pd.to_datetime(air[key]['EndUtc'])
    air[key]['time'] = start + (end - start) / 2

In [ ]:
# air

In [ ]:
# fig, ax = plt.subplots()
# ax.plot(air['time'], air['PN'])
# ax.plot(np.array(times), func(np.array(wind)) * 1.0e4)

In [ ]:
from scipy.ndimage import gaussian_filter
smoothed = gaussian_filter(wind * 1.0e4, sigma=2)
fig, ax = plt.subplots()
ax.plot(air['backersvej']['time'], air['backersvej']['PN'])
ax.plot(times, smoothed)
ax.set_xlim(times[0], times[-1])

In [ ]:
import plotly.graph_objects as go

fig = go.FigureWidget()

# Add traces
for key, df in air.items():
    fig.add_trace(go.Scatter(x=df['time'], y=df['PN'],
                        mode='lines',
                        name=key))
fig.add_trace(go.Scatter(x=times, y=smoothed,
                    mode='lines',
                    name='South wind'))

fig.update_layout(xaxis_range=[times[0], times[-1]])
fig

In [ ]:
# # Make 10th trace visible
# fig.data[10].visible = True

# Create and add slider
steps = []
for im, month in enumerate(months):
    step = dict(
        method="relayout",
        args=[{"xaxis.range": [f'{year}-{im+1:02}-01',
                               f'{year}-{im+1:02}-{days_in_month[month]}']},
              # {"title": "Slider switched to step: " + str(i)}],  # layout attribute
             ],
        label=month
    )
    # step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

    
# print(steps)
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Month: "},
    # pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    height=800,
)

In [ ]:
from plotly.subplots import make_subplots
fig = go.FigureWidget(make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02))

# Add traces
for key, df in air.items():
    fig.add_trace(go.Scatter(x=df['time'], y=df['PN'],
                        mode='lines',
                        name=key),
                 row=1, col=1)
fig.add_trace(go.Scatter(x=times, y=smoothed,
                    mode='lines',
                    name='South wind'),
             row=2, col=1)

fig.update_layout(xaxis_range=[times[0], times[-1]])


# # Make 10th trace visible
# fig.data[10].visible = True

# Create and add slider
steps = []
for im, month in enumerate(months):
    step = dict(
        method="relayout",
        args=[{"xaxis.range": [f'{year}-{im+1:02}-01',
                               f'{year}-{im+1:02}-{days_in_month[month]}']},
              # {"title": "Slider switched to step: " + str(i)}],  # layout attribute
             ],
        label=month
    )
    # step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

    
# print(steps)
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Month: "},
    # pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    height=800,
)
fig

In [ ]:
np.datetime64('2023-03-01')